In [70]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb

%matplotlib inline

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x7fadb90099b0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/xgboost/core.py", line 957, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'
Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x7fadb8e982e8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/xgboost/core.py", line 957, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'
Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x7fadb900d518>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/xgboost/core.py", line 957, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


In [71]:
from google.colab import drive
drive.mount('/content/gdrive/')
%pwd
%cd 'gdrive/My Drive'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive'
/content/gdrive/My Drive


In [72]:
def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

In [73]:
def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
        
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    
    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs


In [74]:
def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

In [12]:
def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num,filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)
    
    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
    return basis

In [75]:
train = pd.read_csv('Train.csv')

def get_audio(file, FFT_size=2048, hop_size=10, sample_rate=44100):
  global audio
  sample_rate, audio = wavfile.read(file)
  audio = audio / np.max(np.abs(audio))
  # hop_size in ms
  audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
  frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
  frame_num = int((len(audio) - FFT_size) / frame_len) + 1
  audio_framed = np.zeros((frame_num,FFT_size))
    
  for n in range(frame_num):
      audio_framed[n] = audio[n*frame_len:n*frame_len+FFT_size]

  hop_size = 15 #ms
  FFT_size = 2048

  window = get_window("hann", FFT_size, fftbins=True)

  audio_win = audio_framed * window

  audio_winT = np.transpose(audio_win)

  audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

  for n in range(audio_fft.shape[1]):
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

  audio_fft = np.transpose(audio_fft)

  #Calculate signal power
  audio_power = np.square(np.abs(audio_fft))

  freq_min = 0
  freq_high = sample_rate / 2
  mel_filter_num = 10

  filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)

  filters = get_filters(filter_points, FFT_size)

  # taken from the librosa library  
  enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
  filters *= enorm[:, np.newaxis]

  #Filter the signal
  audio_filtered = np.dot(filters, np.transpose(audio_power))
  audio_log = 10.0 * np.log10(audio_filtered)

  dct_filter_num = 40

  dct_filters = dct(dct_filter_num, mel_filter_num)

  cepstral_coefficents = np.dot(dct_filters, audio_log)

  return cepstral_coefficents


output = get_audio(train.fn.sample().values[0], FFT_size=2048, hop_size=10, sample_rate=44100)

'audio_files/DLMUVT3.wav'

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)
label = np.random.randint(2, size=40)  # binary target
dtrain = xgb.DMatrix(output, label=label)

In [62]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'logloss'

In [68]:
watchlist = [(dtest, 'test'), (dtrain, 'train')]
num_round = 40
bst = xgb.train(param, dtrain, num_round, watchlist)

XGBoostError: ignored

In [33]:
bst.save_model('0001.model')

In [36]:
import matplotlib
xgb.plot_importance(bst)

XGBoostError: ignored